In this notebook we will see the basics of how to use MiniSom.

Let's start importing MiniSom:

In [ ]:
from minisom import MiniSom

MiniSom relies on the Python ecosystem to import and preprocess the data. For this example we will load the <a href="https://archive.ics.uci.edu/ml/datasets/seeds">seeds</a> dataset dataset using pandas:

In [ ]:
import pandas as pd
import numpy as np
columns=['area', 'perimeter', 'compactness', 'length_kernel', 'width_kernel',
                   'asymmetry_coefficient', 'length_kernel_groove', 'target']
data = pd.read_csv('https://archive.ics.uci.edu/ml/machine-learning-databases/00236/seeds_dataset.txt', 
                    names=columns, 
                   sep='\t+', engine='python')
target = data['target'].values
label_names = {1:'Kama', 2:'Rosa', 3:'Canadian'}
data = data[data.columns[:-1]]
# data normalization
data = (data - np.mean(data, axis=0)) / np.std(data, axis=0)
data = data.values

We can initialize and train MiniSom as follows:

In [ ]:
# Initialization and training
n_neurons = 9
m_neurons = 9
som = MiniSom(n_neurons, m_neurons, data.shape[1], sigma=1.5, learning_rate=.5, 
              neighborhood_function='gaussian', random_seed=0)

som.pca_weights_init(data)
som.train(data, 1000, verbose=True)  # random training

To visualize the result of the training we can plot the distance map (U-Matrix) using a pseudocolor where the neurons of the maps are displayed as an array of cells and the color represents the (weights) distance from the neighbour neurons. On top of the pseudo color we can add markers that repesent the samples mapped in the specific cells:

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

plt.figure(figsize=(9, 9))

plt.pcolor(som.distance_map().T, cmap='bone_r')  # plotting the distance map as background
plt.colorbar()

# Plotting the response for each pattern in the iris dataset
# different colors and markers for each label
markers = ['o', 's', 'D']
colors = ['C0', 'C1', 'C2']
for cnt, xx in enumerate(data):
    w = som.winner(xx)  # getting the winner
    # palce a marker on the winning position for the sample xx
    plt.plot(w[0]+.5, w[1]+.5, markers[target[cnt]-1], markerfacecolor='None',
             markeredgecolor=colors[target[cnt]-1], markersize=12, markeredgewidth=2)

plt.show()

To have an overview of how the samples are distributed across the map a scatter chart can be used where each dot represents the coordinates of the winning neuron. A random offset is added to avoid overlaps between points within the same cell.

In [ ]:
w_x, w_y = zip(*[som.winner(d) for d in data])
w_x = np.array(w_x)
w_y = np.array(w_y)

plt.figure(figsize=(10, 9))
plt.pcolor(som.distance_map().T, cmap='bone_r', alpha=.2)
plt.colorbar()

for c in np.unique(target):
    idx_target = target==c
    plt.scatter(w_x[idx_target]+.5+(np.random.rand(np.sum(idx_target))-.5)*.8,
                w_y[idx_target]+.5+(np.random.rand(np.sum(idx_target))-.5)*.8, 
                s=50, c=colors[c-1], label=label_names[c])
plt.legend(loc='upper right')
plt.grid()
plt.savefig('resulting_images/som_seed.png')
plt.show()

To have an idea of which neurons of the map are activated more often we can create another pseudocolor plot that reflects the activation frequencies:

In [ ]:
plt.figure(figsize=(7, 7))
frequencies = som.activation_response(data)
plt.pcolor(frequencies.T, cmap='Blues') 
plt.colorbar()
plt.show()

Wehn dealing with a supervised problem, one can visualize the proportion of samples per class falling in a specific neuron using a pie chart per neuron:

In [ ]:
import matplotlib.gridspec as gridspec

labels_map = som.labels_map(data, [label_names[t] for t in target])

fig = plt.figure(figsize=(9, 9))
the_grid = gridspec.GridSpec(n_neurons, m_neurons, fig)
for position in labels_map.keys():
    label_fracs = [labels_map[position][l] for l in label_names.values()]
    plt.subplot(the_grid[n_neurons-1-position[1],
                         position[0]], aspect=1)
    patches, texts = plt.pie(label_fracs)

plt.legend(patches, label_names.values(), bbox_to_anchor=(3.5, 6.5), ncol=3)
plt.savefig('resulting_images/som_seed_pies.png')
plt.show()

In [ ]:
import plotly.graph_objects as go

win_map = som.win_map(data)
size=som.distance_map().shape[0]
qualities=np.zeros((size,size))
for position, values in win_map.items():
    qualities[position[0], position[1]] = np.mean(abs(values-som.get_weights()[position[0], position[1]]))

layout = go.Layout(title='qualities')
fig = go.Figure(layout=layout)
fig.add_trace(go.Heatmap(z=qualities, colorscale='Viridis'))
fig.show()

In [ ]:
from plotly.subplots import make_subplots
import math
def showPropertyPlot(som, data, columns):
# plots the distances for each different property
    win_map = som.win_map(data)
    size=som.distance_map().shape[0]
    properties=np.zeros((size*size,2+data.shape[1]))
    i=0
    #A = np.array(size*size)
    for row in range(0,size):
        for col in range(0,size):
            #print(len(values))
            properties[size*row+col,0]=row
            properties[size*row+col,1]=col

    for position, values in win_map.items():
        #print(len(values))
        properties[size*position[0]+position[1],0]=position[0]
        properties[size*position[0]+position[1],1]=position[1]
        properties[size*position[0]+position[1],2:] = np.mean(values, axis=0)
        #print(np.mean(values, axis=0))
        i=i+1

    B = ['row', 'col']
    B.extend(columns)
    properties = pd.DataFrame(data=properties, columns=B)
    #properties=properties.sort_values(by=['row', 'col'])
    #properties
    
    fig = make_subplots(rows=math.ceil(math.sqrt(data.shape[1])), cols=math.ceil(math.sqrt(data.shape[1])), shared_xaxes=False, horizontal_spacing=0.1, vertical_spacing=0.05, subplot_titles=columns, column_widths=None, row_heights=None)
    i=0
    zmin=min(np.min(properties.iloc[:,2:]))
    zmax=max(np.max(properties.iloc[:,2:]))
    for property in columns:
        fig.add_traces(
            [go.Heatmap(z=properties.sort_values(by=['row', 'col'])[property].values.reshape(size,size), zmax=zmax, zmin=zmin, coloraxis = 'coloraxis2')],
            rows=[i // math.ceil(math.sqrt(data.shape[1])) + 1 ],
            cols=[i % math.ceil(math.sqrt(data.shape[1])) + 1 ]
            )
        i=i+1

    for layout in fig.layout:
        #print(layout)
        if layout.startswith('xaxis') or layout.startswith('yaxis'):
            fig.layout[layout].visible=False
            fig.layout[layout].visible=False
        if layout.startswith('coloraxis'):
            fig.layout[layout].cmax=zmax
            fig.layout[layout].cmin=zmin   
            #fig.layout[layout].colorscale='Rainbow'
        if layout.startswith('colorscale'):        
            #print(fig.layout[layout])
            fig.layout[layout]={'diverging':'viridis'}

    fig.update_layout(
        #width=800,
        height=800
    )
    fig.show()
    
showPropertyPlot(som, data, columns[:-1])

In [ ]:
def distributionMap(data, clusters, size, columns, minimum, maximum, plottype='barpolar'):
    spec={"type": "polar"}
    fig = make_subplots(rows=size, cols=size, specs=np.full((size,size),spec).tolist(), shared_yaxes=True, shared_xaxes=True,horizontal_spacing=0, vertical_spacing=0, subplot_titles=None, column_widths=None, row_heights=None)
    categories=columns
    if plottype=='spider':
        for index, row in data.iterrows():
            fig.add_traces(
                [go.Scatterpolargl(
                    r=row['max'],
                    name='max',
                    fillcolor='darkgray',
                    line=dict(color='darkgray'),
                    theta=categories,           
                    opacity=0.5),
                go.Scatterpolargl(
                    r=row['mean'],
                    name='mean',
                    fillcolor='blue',
                    line=dict(color='blue'),
                    theta=categories,           
                    opacity=0.5),
                go.Scatterpolargl(
                    r=row['min'],
                    name='min',
                    fillcolor='red',
                    line=dict(color='red'),
                    theta=categories,           
                    opacity=0.5)],
                rows=[row['row'], row['row'], row['row']],
                cols=[row['col'], row['col'], row['col']]
                )
    else:
        for index, row in data.iterrows():
            fig.add_traces(
                [
                go.Barpolar(                   
                    base=minimum-1,
                    r=row['max']-minimum-1,
                    name='max'+str(index),
                    marker_color="darkgray",
                    theta=categories,           
                    #opacity=1
                 ),
                go.Barpolar(           
                    base=minimum-1,
                    r=row['mean']-minimum-1,
                    name='mean'+str(index),
                    marker_color="blue",
                    theta=categories,           
                    #opacity=1
                 ),
                go.Barpolar(           
                    base=minimum-1,
                    r=row['min']-minimum-1,
                    name='min'+str(index),
                    marker_color="darkred",
                    theta=categories,           
                    #opacity=1
                ) 
                ],
                rows=[row['row'], row['row'], row['row']],
                cols=[row['col'], row['col'], row['col']]
                )        

    if plottype=='spider':
        fig.update_traces(mode='lines', fill='toself')
    for layout in fig.layout:
        if layout.startswith('polar'):
            fig.layout[layout].angularaxis.visible=False
            fig.layout[layout].angularaxis.tickfont.size = 7
            fig.layout[layout].radialaxis.visible=True
            fig.layout[layout].radialaxis.tickfont.size = 10
            fig.layout[layout].barmode='overlay'
            fig.layout[layout].radialaxis.range = [minimum, maximum+1]

    fig.update_layout(
        width=900,
        height=900,
        font_size=9,
        showlegend=False
    )

    fig.show()

In [ ]:
def plotDistributionOfDataInClusters(som, data, columns, plottype='barpolar'): 
    size=som.distance_map().shape[0]
    clusters=res = np.array(np.arange(0,size*size)).reshape(size,size)
    distributionMapData=pd.DataFrame(columns=['col', 'row', 'min', 'mean', 'max'])
    win_map=som.win_map(data)
    for position in win_map.keys():
        winner=win_map[position]
        minima=np.min(winner, axis=0)
        means=np.mean(winner, axis=0)
        maxima=np.max(winner, axis=0)
        row=int(position[0]+1)
        col=int(position[1]+1)
        distributionMapData=distributionMapData.append(
            {'col': col,
             # row needs to be shifted because all the other plots (heatmaps) are shown with row 0 at bottom 
             'row': size-row+1, 
             'min': minima, 
             'mean': means, 
             'max': maxima}, verify_integrity=True, ignore_index=True)

    noClusters=np.max(clusters).item()+1
    clusterData=pd.DataFrame(columns=['col', 'row', 'min', 'mean', 'max'])
    minimum=0
    maximum=0
    for cluster in range(0, noClusters):    
        listOfCluster=distributionMapData[distributionMapData.apply(lambda x: clusters[x.row-1,x.col-1] == cluster, axis=1)]
        if (listOfCluster.size>0):
            minima=listOfCluster['min'].values[0]
            for val in listOfCluster['min'].values:
                minima = np.minimum(val, minima) 

            maxima=listOfCluster['max'].values[0]
            for val in listOfCluster['max'].values:
                maxima = np.maximum(val, maxima) 

            flat_list = []
            for sublist in listOfCluster['mean'].values:
                flat_list.append(sublist.tolist())
                means = np.mean(flat_list, axis=0) 
        clusterData=clusterData.append(
            {'col': cluster-size*(cluster//9)+1,
             # row needs to be switched because all the other plots (heatmaps) are shown with row 0 at bottom 
             'row': (cluster)//9+1, 
             'min': minima, 
             'mean': means, 
             'max': maxima}, verify_integrity=True, ignore_index=True)
        minimum=min(minimum,np.min(minima))
        maximum=max(maximum,np.min(maxima))

    distributionMap(clusterData, clusters, size, columns, minimum, maximum, plottype)

plotDistributionOfDataInClusters(som, data, columns[:-1], plottype='barpolar')

# From Hamel, Brown: Improved Interpretability of the Unified Distance Matrix with Connected Components
## Starburst Gradient visualization
At the core of the implementation is the function find.internal.node which, given the map coordinates of a neural element, will find the corresponding internal node of the associated star. Table I shows the pseudo code for this function. Given a position on the map this function first searches the adjacent nodes for the minimal UMAT value using the function find.min. If an adjacent node with a smaller UMAT value than the value of our current node exists and if this UMAT value is smaller than the UMAT values of all other adjacent nodes, then that node lies along the maximum gradient of the surface and we make this node our new current position. If no such node exists, then the gradient at our current position is zero and we are at an internal node.

In [ ]:
def findMin(x, y, umat):
    newxmin=max(0,x-1)
    newxmax=min(umat.shape[0],x+2)
    newymin=max(0,y-1)
    newymax=min(umat.shape[1],y+2)
    minx, miny = np.where(umat[newxmin:newxmax,newymin:newymax] == umat[newxmin:newxmax,newymin:newymax].min())
    return newxmin+minx[0], newymin+miny[0]

def findInternalNode(x, y, umat):
    minx, miny = findMin(x,y,umat)
    if (minx == x and miny == y):
        cx = minx
        cy = miny
    else:
        cx,cy = findInternalNode(minx,miny,umat)
    return cx, cy

In [ ]:
def plotStarburstMap(som):
    layout = go.Layout(title='starburstMap')
    fig = go.Figure(layout=layout)
    fig.add_trace(go.Heatmap(z=som.distance_map()))
    shapes=[]

    for row in np.arange(som.distance_map().shape[0]):
        for col in np.arange(som.distance_map().shape[1]):
            cx,cy = findInternalNode(row, col, som.distance_map())
            shape=go.layout.Shape(
                    type="line",
                    x0=col,
                    y0=row,
                    x1=cy,
                    y1=cx,
                    line=dict(
                        color="Black",
                        width=1
                    )
                )
            shapes=np.append(shapes, shape)

    fig.update_layout(shapes=shapes.tolist()) 
    fig.show()
    
plotStarburstMap(som)

To understand how the training evolves we can plot the quantization and topographic error of the SOM at each step. This is particularly important when estimating the number of iterations to run:

In [ ]:
som = MiniSom(10, 20, data.shape[1], sigma=3., learning_rate=.7,
              neighborhood_function='gaussian', random_seed=10)

max_iter = 1000
q_error = []
t_error = []

for i in range(max_iter):
    rand_i = np.random.randint(len(data))
    som.update(data[rand_i], som.winner(data[rand_i]), i, max_iter)
    q_error.append(som.quantization_error(data))
    t_error.append(som.topographic_error(data))

plt.plot(np.arange(max_iter), q_error, label='quantization error')
plt.plot(np.arange(max_iter), t_error, label='topographic error')
plt.ylabel('quantization error')
plt.xlabel('iteration index')
plt.legend()
plt.show()

Notice that in the snippet above weto run each learning iteration in a for loop and saved the errors in separated lists.